<img src="https://ww2.ufps.edu.co/public/archivos/elementos_corporativos/logoufps.png" style="float: left; margin: 20px; height: 55px">  

<b> <font size='5'>  
HYBRID MODEL BASED ON ARTIFICIAL INTELLIGENCE FOR ENERGY DISPATCH IN A MICROGRID</font>  

<font size='4'> CAISO Data Acquisition Notebook</font> </b>

<font size='3'>  
&nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; 2023<br>
</font>

---

In [1]:
import os.path
from calendar import monthrange, weekday
import csv
import pandas as pd
import numpy as np

In [2]:
data_time = []
data_season = []
data_day_week = []
data_productionPV = []
data_productionWind = []
data_demand = []


def add_to_data(anio,mes,diames,diasemana,produccionFV,produccionWind,demanda):
    
    for hour in range(0,288):
        
        mes = int(mes)
        diames = int(diames)
        
        if(mes>=1 and mes<=3):             #Invierno: 1, Primavera: 2, Verano: 3, Otoño: 4
            data_season.append(1)
        elif(mes>=4 and mes<=6):
            data_season.append(2)
        elif(mes>=7 and mes<=9):
            data_season.append(3)
        elif(mes>=10 and mes<=12):
            data_season.append(4)
        data_day_week.append(diasemana)
        
        hours, count = 0, 0
        for i in range (0,hour):
            count += 1
            if count >= 12:
                hours += 1
                count = 0

        if count == 0:
            minutes = '00'
        else:
            minutes = count*5
        
        if mes<10: mes = "0"+str(mes)
        if diames<10: diames = "0"+str(diames)
        if hours < 10: hours = "0"+str(hours)
        if int(minutes)==5: minutes = "0"+str(minutes)
        
        data_time.append(str(anio)+"-"+str(mes)+"-"+str(diames)+"-T"+str(hours)+":"+str(minutes))
        
        hours = int(hours)
        minutes = int(minutes)
        
        if hour >= len(produccionFV) or hour >= len(produccionWind) or hour >= len(demanda):
            data_productionPV.append(produccionFV[len(produccionFV)-1])
            data_productionWind.append(produccionWind[len(produccionWind)-1])
            data_demand.append(demanda[len(demanda)-1])
        else:
            data_productionPV.append(produccionFV[hour])
            data_productionWind.append(produccionWind[hour])
            data_demand.append(demanda[hour])

In [3]:
def read_the_data(file):
    
    reader = csv.reader(open(file))
    
    temp = []
    for row in reader:
        temp.append(row)
    production_0 = temp[2] #Producción Eólica
    production_1 = temp[1] #Producción FV
    production_0.pop(0)
    production_1.pop(0)
    
    production_hourly_PV = []
    production_hourly_Wind = []
    
    for i in range(0,len(production_1)):
        production_hourly_PV.append(production_1[i])
    
    for i in range(0,len(production_0)):
        production_hourly_Wind.append(production_0[i])
    
    return production_hourly_PV, production_hourly_Wind

In [4]:
def read_the_demand(file):
    
    reader = csv.reader(open(file))
    
    temp = []
    for row in reader:
        temp.append(row)
    demand_0 = temp[3] #Demanda
    demand_0.pop(0)
    
    demand_hourly = []
    
    for i in range(0,len(demand_0)):
        demand_hourly.append(demand_0[i])
    
    return demand_hourly

In [5]:
for year in range(2019,2022):       #Usando datos desde 2019 hasta 2021
    for month in range(1,13):
        if month <= 9: month = "0" + str(month)
        if os.path.isdir("./Produccion/Datos_produccion/"+str(year)+"/"+str(month)):
            for day in range(1,monthrange(year,int(month))[1]+1):
                if day <= 9: day = "0" + str(day)
                if os.path.isfile("./Produccion/Datos_produccion/"+str(year)+"/"+str(month)+"/CAISO-renewables-"+str(year)+str(month)+str(day)+".csv"):
                    file_path = "./Produccion/Datos_produccion/"+str(year)+"/"+str(month)+"/CAISO-renewables-"+str(year)+str(month)+str(day)+".csv"
                    production_PV, production_Wind = read_the_data(file_path)
                    demand_path = "./Consumo/Datos_consumo/"+str(year)+"/"+str(month)+"/CAISO-demand-"+str(year)+str(month)+str(day)+".csv"
                    demand = read_the_demand(demand_path)
                    add_to_data(year,int(month),int(day),weekday(year,int(month),int(day)),production_PV,production_Wind,demand)

In [6]:
def read_extra_data(file):
    
    reader = csv.reader(open(file))
    
    temp = []

    for row in reader:
        temp.append(row)
    
    temp.pop(0)
    temp.pop(0)
    temp.pop(0)

    dhi = []
    dni = []
    Wspeed = []
    humidity = []
    temperature = []
    ghi = []
    
    
    for position in range(0,len(temp)):

        dhi.append(temp[position][5]) 
        dni.append(temp[position][6]) 
        Wspeed.append(temp[position][7])
        humidity.append(temp[position][8]) 
        temperature.append(temp[position][9])
        ghi.append(temp[position][10]) 

    return dhi, dni, Wspeed, humidity, temperature, ghi

In [7]:
dhi = []
dni = []
Wspeed = []
humidity = []
temperature = []
ghi = []

for city in ["Baker","Bishop","Fortuna","Sacramento","SanDiego"]:
    for year in range(2019,2022):
        
        data1, data2, data3, data4, data5, data6 = read_extra_data("./Datos_extra/"+city+"_NREL_data_"+str(year)+".csv")
        
        dhi += data1
        dni += data2
        Wspeed += data3
        humidity += data4
        temperature += data5
        ghi += data6

    if city=="Baker":
        baker=[dhi,dni,Wspeed,humidity,temperature,ghi]
    elif city=="Bishop":
        bishop=[dhi,dni,Wspeed,humidity,temperature,ghi]
    elif city=="Fortuna":
        fortuna=[dhi,dni,Wspeed,humidity,temperature,ghi]
    elif city=="Sacramento":
        sacramento=[dhi,dni,Wspeed,humidity,temperature,ghi]
    else:
        sandiego=[dhi,dni,Wspeed,humidity,temperature,ghi]
    
    dhi = []
    dni = []
    Wspeed = []
    humidity = []
    temperature = []
    ghi = []

for position in range(0,len(baker[0])):
    dhi.append((float(baker[0][position])+float(bishop[0][position])+float(fortuna[0][position])+float(sacramento[0][position])+float(sandiego[0][position]))/5)
    dni.append((float(baker[1][position])+float(bishop[1][position])+float(fortuna[1][position])+float(sacramento[1][position])+float(sandiego[1][position]))/5)
    Wspeed.append((float(baker[2][position])+float(bishop[2][position])+float(fortuna[2][position])+float(sacramento[2][position])+float(sandiego[2][position]))/5)
    humidity.append((float(baker[3][position])+float(bishop[3][position])+float(fortuna[3][position])+float(sacramento[3][position])+float(sandiego[3][position]))/5)
    temperature.append((float(baker[4][position])+float(bishop[4][position])+float(fortuna[4][position])+float(sacramento[4][position])+float(sandiego[4][position]))/5)
    ghi.append((float(baker[5][position])+float(bishop[5][position])+float(fortuna[5][position])+float(sacramento[5][position])+float(sandiego[5][position]))/5)


In [8]:
produccion_energia = pd.DataFrame(data={'Time':data_time,'Estacion':data_season,'Dia semana':data_day_week,'DHI':dhi,'DNI':dni,'Velocidad':Wspeed,'Humedad':humidity,'Temperatura':temperature,'GHI':ghi,'Produccion FV (MW)':data_productionPV,'Produccion Eolica (MW)':data_productionWind,'Demanda (MW)':data_demand})

produccion_energia.replace('', np.nan, inplace=True)

produccion_energia['Produccion FV (MW)'].fillna(method='ffill', inplace=True)
produccion_energia['Produccion Eolica (MW)'].fillna(method='ffill', inplace=True)
produccion_energia['Demanda (MW)'].fillna(method='ffill', inplace=True)

produccion_energia['Produccion FV (MW)'].astype('int')
produccion_energia['Produccion Eolica (MW)'].astype('int')
produccion_energia['Demanda (MW)'].astype('int')

print(produccion_energia)

produccion_energia.to_csv('3_Database.csv')
#produccion_energia.to_excel('produccion_energia_database.xlsx')

                     Time  Estacion  Dia semana  DHI  DNI  Velocidad  Humedad  \
0       2019-01-01-T00:00         1           1  0.0  0.0       2.88   56.036   
1       2019-01-01-T00:05         1           1  0.0  0.0       2.88   56.036   
2       2019-01-01-T00:10         1           1  0.0  0.0       2.88   56.194   
3       2019-01-01-T00:15         1           1  0.0  0.0       2.88   56.344   
4       2019-01-01-T00:20         1           1  0.0  0.0       2.84   56.440   
...                   ...       ...         ...  ...  ...        ...      ...   
315643  2021-12-31-T23:35         4           4  0.0  0.0       1.64   64.898   
315644  2021-12-31-T23:40         4           4  0.0  0.0       1.64   64.970   
315645  2021-12-31-T23:45         4           4  0.0  0.0       1.64   65.042   
315646  2021-12-31-T23:50         4           4  0.0  0.0       1.64   65.234   
315647  2021-12-31-T23:55         4           4  0.0  0.0       1.62   65.308   

        Temperatura  GHI Pr